** 预训练模型特征提取和重新训练 **
* 参考 https://zhuanlan.zhihu.com/p/25978105

In [50]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_inception_resnet_v2.h5"]:
    print(filename)
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

gap_ResNet50.h5
gap_Xception.h5
gap_inception_resnet_v2.h5


In [51]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [52]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 3s - loss: 0.0567 - acc: 0.9810 - val_loss: 0.0172 - val_acc: 0.9960
Epoch 2/8
20000/20000 [==============================] - 2s - loss: 0.0200 - acc: 0.9940 - val_loss: 0.0119 - val_acc: 0.9978
Epoch 3/8
20000/20000 [==============================] - 2s - loss: 0.0154 - acc: 0.9955 - val_loss: 0.0098 - val_acc: 0.9982
Epoch 4/8
20000/20000 [==============================] - 2s - loss: 0.0133 - acc: 0.9963 - val_loss: 0.0106 - val_acc: 0.9964
Epoch 5/8
20000/20000 [==============================] - 2s - loss: 0.0109 - acc: 0.9964 - val_loss: 0.0083 - val_acc: 0.9980
Epoch 6/8
20000/20000 [==============================] - 2s - loss: 0.0096 - acc: 0.9967 - val_loss: 0.0076 - val_acc: 0.9982
Epoch 7/8
20000/20000 [==============================] - 2s - loss: 0.0095 - acc: 0.9970 - val_loss: 0.0075 - val_acc: 0.9982
Epoch 8/8
20000/20000 [==============================] - 2s - loss: 0

In [54]:
model.save('model_bottleneck.h5')

In [55]:
print(X_test.shape)
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

(12500, 6240)
11456/12500 [==========================>...] - ETA: 0s

In [56]:
from keras.preprocessing.image import *

y_train=model.predict(X_train, verbose=1)
for i,v in enumerate(y_train):
    if v[0]>0.5:
        y_train[i]=1
    else:
        y_train[i]=0
print(y_train)
image_size = (299, 299)
gen = ImageDataGenerator()
generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)
for i, fname in enumerate(generator.filenames):
    index = fname[fname.rfind('/')+1:fname.rfind('.')-2]
    if(index=='cat' and y_train[i]!=0):
        print(i,fname)
        
print('.......')    

24992/25000 [============================>.] - ETA: 0s[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 1.]
 [ 0.]
 [ 0.]]
Found 25000 images belonging to 2 classes.
(3612, 'cat/cat.2.jpg')
(4723, 'cat/cat.3.jpg')
(8056, 'cat/cat.6.jpg')
(9167, 'cat/cat.7.jpg')
(11389, 'cat/cat.9.jpg')
.......


In [84]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

image_size = (299, 299)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('new.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
